In [53]:
#!/usr/bin/env python3
"""
Example usage of the ODrive python library to monitor and control ODrive devices
"""

from __future__ import print_function
import odrive
from odrive.enums import *
import time
import math

# Find a connected ODrive (this will block until you connect one)
print("finding an odrive...")
odrv = odrive.find_any()

# 2)Apply new config
print("applying config...")
odrv.config.dc_bus_overvoltage_trip_level = 30
odrv.config.dc_max_positive_current = 3
odrv.config.dc_max_negative_current = -3
odrv.axis0.config.motor.motor_type = MotorType.HIGH_CURRENT
odrv.axis0.config.motor.torque_constant = 0.05513333333333333
odrv.axis0.config.motor.pole_pairs = 7
odrv.axis0.config.motor.current_soft_max = 70
odrv.axis0.config.motor.current_hard_max = 90
odrv.axis0.config.motor.calibration_current = 10
odrv.axis0.config.motor.resistance_calib_max_voltage = 2
odrv.axis0.config.torque_soft_min = -0.7718666666666667
odrv.axis0.config.torque_soft_max = 0.7718666666666667
odrv.axis0.config.calibration_lockin.current = 10
odrv.axis0.controller.config.input_mode = InputMode.PASSTHROUGH
odrv.axis0.controller.config.control_mode = ControlMode.VELOCITY_CONTROL
odrv.axis0.controller.config.vel_limit = 10
odrv.axis0.controller.config.vel_limit_tolerance = 15
odrv.inc_encoder0.config.cpr = 8192
odrv.inc_encoder0.config.enabled = True
odrv.config.gpio7_mode = GpioMode.DIGITAL
odrv.axis0.commutation_mapper.config.use_index_gpio = True
odrv.axis0.commutation_mapper.config.index_gpio = 7
odrv.axis0.pos_vel_mapper.config.use_index_gpio = True
odrv.axis0.pos_vel_mapper.config.index_gpio = 7
odrv.axis0.pos_vel_mapper.config.index_offset = 0
odrv.axis0.pos_vel_mapper.config.index_offset_valid = True
odrv.axis0.config.load_encoder = EncoderId.INC_ENCODER0
odrv.axis0.config.commutation_encoder = EncoderId.INC_ENCODER0

# Calibrate motor and wait for it to finish
print("starting calibration...")
odrv.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
while odrv.axis0.current_state != AxisState.IDLE:
    time.sleep(0.1)


finding an odrive...
applying config...
starting calibration...


In [10]:
#anticogging
odrv.axis0.controller.config.vel_gain = 0.5
odrv.axis0.config.anticogging.max_torque = 0.15
odrv.axis0.config.anticogging.calib_start_vel = 0.5
odrv.axis0.config.anticogging.calib_end_vel = 0.15
odrv.axis0.config.anticogging.calib_coarse_integrator_gain = 10000

odrv.axis0.requested_state = 14
# for some reason ANTICOGGING_CALIBRATION is not in AxisState, but it is on the odrive firmware
while odrv.axis0.current_state != AxisState.IDLE:
    time.sleep(0.1)
    
from fibre.libfibre import ObjectLostError
print("Connecting to ODrive")
odrv = odrive.find_any()
print("S/N: %d" % odrv.serial_number)

print("Rebooting ODrive...")
try:
    odrv.save_configuratoin() # or save_configuration() or erase_configuration()
except ObjectLostError:
    pass # yeah, the object disappeared, I know

print("Reconnecting to ODrive")
odrv = odrive.find_any()
print("S/N: %d" % odrv.serial_number)


In [63]:
#odrv.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
odrv.axis0.controller.input_vel = -0.8
time.sleep(10)
odrv.axis0.controller.input_vel = 0

In [62]:
multiplier = 1

In [55]:
odrv.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

odrv.axis0.controller.config.vel_limit = 15
max_speed = 8
increment = 2

distance = 0
time_tot = 0

for i in range(10):
    start_abs = time.time()
    #CALCULATE DISTANCE
    for i in range(1,int(max_speed*increment+1)):
        start = time.time()
        speed = (i/increment)*multiplier
        odrv.axis0.controller.input_vel = speed

        time.sleep(0.1)
        timediff = time.time() -start
        distance += timediff*speed
        time_tot += timediff

    start = time.time()
    for i in range(1,int(max_speed*increment+1)):
        start = time.time()
        speed = (max_speed - i/increment)*multiplier
        odrv.axis0.controller.input_vel = speed

        time.sleep(0.1)
        timediff = time.time() - start
        distance += timediff*speed
        time_tot += timediff

    print("Distance traveled: "+ str(distance*0.078*3.1415) + "m")
    print("Calculated time: "+ str(time_tot) + "s")
    print("Actual time: " + str(time.time()-start_abs))
    multiplier = -multiplier
    time.sleep(0.5)



Distance traveled: 3.204192840097189m
Calculated time: 3.270416259765625s
Actual time: 3.271165132522583
Distance traveled: 0.011084228747606277m
Calculated time: 6.535645961761475s
Actual time: 3.2658333778381348
Distance traveled: 3.2151936723343133m
Calculated time: 9.805376768112183s
Actual time: 3.2703967094421387
Distance traveled: 0.007175722026586532m
Calculated time: 13.079061031341553s
Actual time: 3.274379253387451
Distance traveled: 3.2111188691740042m
Calculated time: 16.34917712211609s
Actual time: 3.2708511352539062
Distance traveled: 0.020585880912065507m
Calculated time: 19.606882572174072s
Actual time: 3.258270263671875
Distance traveled: 3.2177793438981777m
Calculated time: 22.86925172805786s
Actual time: 3.263021230697632
Distance traveled: 0.010458652681589128m
Calculated time: 26.142157077789307s
Actual time: 3.273594856262207
Distance traveled: 3.2130198422420024m
Calculated time: 29.41051197052002s
Actual time: 3.268967390060425
Distance traveled: 0.011827611513